In [40]:
import tensorflow as tf
from keras import datasets, layers, models, optimizers
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, Callback

In [ ]:
# Клас Callback для виведення результатів навчання після закінчення
class PrintResultsCallback(Callback):
    def on_train_end(self, logs=None):
        train_loss, train_accuracy = self.model.evaluate(x_train, y_train, verbose=0)
        test_loss, test_accuracy = self.model.evaluate(x_test, y_test, verbose=0)
        print(f"Точність на навчальних данних: {train_accuracy}, втрата на навчальних данних: {train_loss}")
        print(f"Точність на валідаційних данних: {test_accuracy}, втрата на валідаційних данних: {test_loss}")

In [41]:
# Завантаження даних CIFAR-10
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

In [42]:
# Конвертація цілих чисел в числа з плаваючою комою; нормалізація та центрування
x_train = x_train.astype("float32")  
x_test = x_test.astype("float32")
mean = np.mean(x_train)
std = np.std(x_train)
x_test = (x_test - mean) / std
x_train = (x_train - mean) / std

In [43]:
# Аугментація даних
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)
datagen.fit(x_train)

In [44]:

# Перетворення міток в формат one-hot
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Назви міток
class_names = {
    0: "Літак",
    1: "Автомобіль",
    2: "Птах",
    3: "Кіт",
    4: "Олень",
    5: "Собака",
    6: "Жаба",
    7: "Кінь",
    8: "Корабель",
    9: "Вантажівка"
}


In [45]:
# Функція для побудови моделі
def build_model():
    model = models.Sequential()
    model.add(layers.BatchNormalization(input_shape=x_train.shape[1:]))
    model.add(layers.Conv2D(64, (5, 5), padding="same", activation="elu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.BatchNormalization(input_shape=x_train.shape[1:]))
    model.add(layers.Conv2D(128, (5, 5), padding="same", activation="elu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.BatchNormalization(input_shape=x_train.shape[1:]))
    model.add(layers.Conv2D(256, (5, 5), padding="same", activation="elu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.Flatten())
    model.add(layers.Dense(256))
    model.add(layers.Activation("elu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10))
    model.add(layers.Activation("softmax"))
    return model

# Побудова моделі
model = build_model()

# Компіляція моделі
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Виведення опису моделі
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ batch_normalization_6                │ (None, 32, 32, 3)           │              12 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 32, 32, 64)          │           4,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 16, 16, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 16, 16, 128)         │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 8, 8, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 8, 8, 256)           │         819,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │       1,048,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │           2,570 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_5 (Activation)            │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,081,430 (7.94 MB)

 Trainable params: 2,081,040 (7.94 MB)

 Non-trainable params: 390 (1.52 KB)

In [46]:
# Опціонально: Визначення зупинки раннього навчання
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Навчання моделі з використанням зупинки раннього навчання та виведенням результатів
history = model.fit(datagen.flow(x_train, y_train, batch_size=128),
                    epochs=150, validation_data=(x_test, y_test), callbacks=[early_stopping, PrintResultsCallback()])

Epoch 1/150
391/391 ━━━━━━━━━━━━━━━━━━━━ 389s 961ms/step - accuracy: 0.2424 - loss: 2.7771 - val_accuracy: 0.4731 - val_loss: 1.4891
Epoch 2/150
391/391 ━━━━━━━━━━━━━━━━━━━━ 360s 915ms/step - accuracy: 0.3924 - loss: 1.6946 - val_accuracy: 0.5717 - val_loss: 1.2871
Epoch 3/150
391/391 ━━━━━━━━━━━━━━━━━━━━ 372s 946ms/step - accuracy: 0.5153 - loss: 1.3572 - val_accuracy: 0.6228 - val_loss: 1.0734
Epoch 4/150
391/391 ━━━━━━━━━━━━━━━━━━━━ 362s 920ms/step - accuracy: 0.5739 - loss: 1.2095 - val_accuracy: 0.6791 - val_loss: 0.9762
Epoch 5/150
391/391 ━━━━━━━━━━━━━━━━━━━━ 371s 944ms/step - accuracy: 0.6071 - loss: 1.1120 - val_accuracy: 0.6698 - val_loss: 0.9933
Epoch 6/150
391/391 ━━━━━━━━━━━━━━━━━━━━ 370s 942ms/step - accuracy: 0.6371 - loss: 1.0403 - val_accuracy: 0.7180 - val_loss: 0.8412
Epoch 7/150
391/391 ━━━━━━━━━━━━━━━━━━━━ 364s 924ms/step - accuracy: 0.6562 - loss: 0.9848 - val_accuracy: 0.7147 - val_loss: 0.8542
Epoch 8/150
391/391 ━━━━━━━━━━━━━━━━━━━━ 336s 854ms/step - accuracy: 

In [1]:
# Відображення графіків точності та втрат
plt.plot(history.history['accuracy'], label='Точність на навчальному наборі')
plt.plot(history.history['val_accuracy'], label='Точність на перевірочному наборі')
plt.xlabel('Епохи')
plt.ylabel('Точність')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Втрати на навчальному наборі')
plt.plot(history.history['val_loss'], label='Втрати на перевірочному наборі')
plt.xlabel('Епохи')
plt.ylabel('Втрати')
plt.legend()
plt.show()

# Оцінка точності на тестовому наборі даних
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Точність класифікації на тестовому наборі даних:', test_acc)

# Відображення деяких прикладів передбачення моделі
predictions = model.predict(x_test)
num_images = 5
plt.figure(figsize=(10, 10))
for i in range(num_images):
    plt.subplot(1, num_images, i+1)
    plt.imshow(x_test[i])
    predicted_label = class_names[np.argmax(predictions[i])]
    true_label = class_names[np.argmax(y_test[i])]
    plt.title(f'Predicted: {predicted_label}\nTrue: {true_label}')
    plt.axis('off')
plt.show()

NameError: name 'plt' is not defined